In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [4]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("data/en_ewt-up-test.conllu", "test.json")

In [5]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'train.json', 'test': 'test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['predicate', 'predicate_position', 'word', 'arguments'],
        num_rows: 40482
    })
    test: Dataset({
        features: ['predicate', 'predicate_position', 'word', 'arguments'],
        num_rows: 4799
    })
})

In [6]:
datasets["train"][0]

{'predicate': 'kill.01',
 'predicate_position': 7,
 'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.'],
 'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_']}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,predicate,predicate_position,word,arguments
0,know.01,2,"[I, KNOW, YOU, HAVE, A, LOT, GOING, ON, WITH, YOU, AND, THE, LITTLE, WOMAN, .]","[ARG0, V, _, ARG1, _, _, _, _, _, _, _, _, _, _, _]"
1,provide.01,9,"[Parties, ,, attached, is, the, promised, ruling, that, provides, procedural, guidance, for, the, hearings, on, 12/27, and, 12/28, .]","[_, _, _, _, _, _, ARG0, R-ARG0, V, _, ARG1, _, _, _, _, ARGM-TMP, _, _, _]"
2,deworm.01,6,"[They, made, need, medication, or, deworming, or, something, .]","[_, _, _, _, _, V, _, _, _]"
3,provide.01,47,"[It, came, from, the, nation, whose, king, Enrique, VIII, adulterated, the, Bible, so, that, divorce, could, be, allowed, and, in, this, way, be, able, to, give, loose, rein, to, the, many, divorces, from, his, wives, and, subsequent, murdering, of, the, same, ones, and, to, whom, God, provided, a, wife, with, six, fingers, as, abomination, (, Anne, Boleyn, ), ..., once, again, 6, ,, number, of, the, Beast, ....]","[_, _, _, _, ARG2, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, R-ARG2, ARG0, V, _, ARG1, _, _, _, _, ARGM-PRD, _, C-ARG1, _, _, _, _, _, _, _, _, _, _, _, _]"
4,choose.01,24,"[The, Sunburn, missile, ,, with, its, incredible, speed, and, ability, to, avoid, radar, detection, ,, would, do, terrible, damage, these, ships, if, Iran, chooses, to, retaliate, in, the, Gulf, after, an, American, attack, within, its, borders, .]","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG0, V, _, ARG1, _, _, _, _, _, _, _, _, _, _, _]"
5,hope.01,4,"[But, let, s, hope, for, their, sake, (, and, the, sake, of, all, space, lovers, out, there, ), that, they, can, redefine, their, image, and, rekindle, the, hope, of, space, colonization, again, .]","[_, _, ARG0, V, _, _, ARGM-PRP, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG1, _, _, _, _, _, _, _, _, _, _, _]"
6,be.01,48,"[The, best, option, ,, now, ,, under, the, present, difficult, circumstances, is, for, Colombo, to, do, its, own, dirty, work, ,, although, New, Delhi, can, always, be, counted, on, to, render, good, neighborly, help, because, of, the, shared, belief, that, religion, ,, ethnicity, and, language, can, not, be, the, basis, for, secession, .]","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG1, _, _, _, _, ARGM-MOD, ARGM-NEG, V, _, ARG2, _, _, _]"
7,control.01,52,"[Goal, is, to, (, politely, ?, ), refute, Loretta, Lynch, 's, and, Carl, Woods, ', continued, assertions, that, 1, ), California, 's, move, to, deregulate, was, based, solely, on, ideology, with, no, basis, in, fact, and, 2, ), the, solution, is, to, turn, back, the, clock, to, command, -, and, -, control, regulation, .]","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _]"
8,house.01,12,"[What, will, the, federal, government, do, to, prevent, inflation, in, the, housing, and, building, market, ?]","[_, _, _, _, _, _, _, _, _, _, _, V, _, _, _, _]"
9,recommend.01,4,"[I, will, never, recommend, this, gym, to, any, woman, .]","[ARG0, ARGM-MOD, ARGM-NEG, V, _, ARG1, _, _, ARG2, _]"


### Tokenization

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
def align(tokenized_input, labels):
    word_ids = tokenized_input.word_ids()
    aligned_labels = ['_'] * len(word_ids) 
    label_index = 0 
    for i, word_id in enumerate(word_ids):
        try:
            if word_id is None:
                aligned_labels[i] = '[PAD]'
                continue 
            original_label = labels[word_id]
            if original_label == '_':
                continue 
            if i == 0 or word_id != word_ids[i-1]:
                prefix = ''
            else:
                prefix = ''
            aligned_labels[i] = f'{prefix}{original_label}'
        except:
            pass

In [13]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = '[PAD]'
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])
    tokenized_input['labels'] = aligned_labels
    
    return tokenized_input


In [49]:
print(len(datasets['train']))
training_set = [tokenize(instance) for instance in datasets['train']]
test_set = [tokenize(instance) for instance in datasets['test']]


40482


In [50]:
sample = training_set[0]
print(sample)

{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': ['[PAD]', '_', '_', '_', '_', '_', '_', 'ARG0', 'V', 'ARG1', 'ARG1', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'ARGM-LOC', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']}


In [51]:
from datasets import Dataset, Features, Sequence, Value

features = Features({
    "input_ids": Sequence(feature=Value(dtype='int64')),
    "attention_mask": Sequence(feature=Value(dtype='int64')),
    "labels": Sequence(feature=Value(dtype='int64')),
})

In [52]:
unseen = ['R-ARGM-ADJ', '_', 'ARGM-ADJ']
label_list = list(set(label for instance in training_set for label in instance['labels'] + unseen))
label_dict = {label:int(i) for i, label in enumerate(list(label_list))}
# label_dict['[PAD]'] = -100
for instance in training_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]
for instance in test_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]

In [18]:
dataset_train = Dataset.from_dict({"input_ids": [item['input_ids'] for item in training_set],
                             "attention_mask": [item['attention_mask'] for item in training_set],
                             "labels": [item['labels'] for item in training_set]}, features=features)

dataset_test = Dataset.from_dict({"input_ids": [item['input_ids'] for item in test_set],
                             "attention_mask": [item['attention_mask'] for item in test_set],
                             "labels": [item['labels'] for item in test_set]}, features=features)

In [20]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

## Base model

In [22]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [23]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [24]:
metric = load_metric("seqeval")


C:\Users\katko\AppData\Local\Temp\ipykernel_30772\2951164050.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
labels = list(label_list)
metric.compute(predictions=[labels], references=[labels])

C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARG4 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C-ARGM-GOL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\se

{'ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1-DSP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG4': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CAU': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-COM': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CXN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-DIR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-EXT': {'precisi

In [26]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Training the basic model

In [ ]:
import pickle
def load_model(pickles):
    '''
    Input: string name with .pickle extension. Must be present in working directory. Check with 'pwd' command
    Loads pretrained model variable from pickle file at filepath(pickle).
    '''
    with open(pickles, 'rb') as p:
        return pickle.load(p)

### Current version = opt 2

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, DistilBertForTokenClassification
if input("Type 'load' to load a model from the directory, else press enter") == 'load':
    # basic_model = load_model('basic_bert.pickle')
    opt = input("Type 1 to load from Jing's repo, type 2 to load from Demothi's repo")
    if opt == '1':
        basic_model = DistilBertForTokenClassification.from_pretrained("Jing1113/distilbert-base-uncased-finetuned-srl")
    elif opt == '2':
        basic_model = DistilBertForTokenClassification.from_pretrained("somskat/distilbert-base-uncased-finetuned-ner")

else:
    basic_model = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased')


In [ ]:
trainer = Trainer(
    basic_model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
if input("type 'y' to train model, press enter to skip") == 'y':
    trainer.train()

## Evaluation

In [ ]:
import torch
x, y = dataset_test.select_columns(['input_ids','attention_mask']), dataset_test.select_columns(['labels'])
# basic_model.eval()
# with torch.no_grad():
predictions, labels, _ = trainer.predict(x)
# predictions = np.argmax(predictions, axis=2)



In [ ]:
predictions, labels, _ = trainer.predict(dataset_test)

In [ ]:
predictions = np.argmax(predictions, axis=2)

In [ ]:
predictions[0]

In [ ]:
labels[0]

In [ ]:
flat_predictions = [pred for sublist in predictions for pred in sublist]
flat_labels = [label for sublist in labels for label in sublist]

#Now create the DataFrame
df = pd.DataFrame({
    'predictions': flat_predictions,
    'true_labels': flat_labels
})

df.to_csv('test_set_predictions.csv', index=False)

### Save Model

In [ ]:
trainer.push_to_hub()


In [ ]:
import pickle
def picklify(pickles,not_so_pickles):
    '''
    Saves a model or variable to a pickle file 
    '''
    with open(pickles, 'wb') as p:
        pickle.dump(not_so_pickles, p)
if input("Save to pickle file?(y/n)") == 'y': 
    picklify('basic_bert.pickle',model)

# Advanced Model

### Replace -100 labels with 0 to ensure that the CRF can handle it

In [53]:
import pandas as pd
df_train,  df_test = pd.DataFrame(dataset_train), pd.DataFrame(dataset_test)
df_test['labels'], df_train['labels'] = df_test['labels'].apply(lambda x: [e if e != -100 else 61 for e in x]), df_train['labels'].apply(lambda x: [e if e != -100 else 0 for e in x])

## Create generator for training advanced model

In [60]:
advanced_train, advanced_test = pd.DataFrame(training_set), pd.DataFrame(test_set)
advanced_train

,attention_mask,input_ids,labels
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2632, 1011, 23564, 2386, 1024, 2137, 274...","[40, 59, 59, 59, 59, 59, 59, 48, 13, 3, 3, 59,..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 13, 59, 59, 59, 3, 59, 59, 59, 59..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 13, 59, 5..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 48, 59, 59, 59, 59, 10, 59, 13, 2..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5..."
...,...,...,...
40477,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 3, 10, 50, 13, 18, 8, 59, 59, 59, 59, 59,..."
40478,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 13, 5..."
40479,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 48, 59, 59, 59, 59, 59, 59, 59, 8, 37, 59..."
40480,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5..."


In [61]:
def find_predicate(tokens, labels):
    split = tokens.index(102)
    return tokens[:split], tokens[split+1:-1], labels[:split],

a,b,c = find_predicate(advanced_train.at[1,'input_ids'],advanced_train.at[1,'labels'])

In [63]:
advanced_train['predicate'] = advanced_train['input_ids']
advanced_test['predicate'] = advanced_test['input_ids']

In [64]:
advanced_train

,attention_mask,input_ids,labels,predicate
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2632, 1011, 23564, 2386, 1024, 2137, 274...","[40, 59, 59, 59, 59, 59, 59, 48, 13, 3, 3, 59,...","[101, 2632, 1011, 23564, 2386, 1024, 2137, 274..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 13, 59, 59, 59, 3, 59, 59, 59, 59...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 13, 59, 5...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 48, 59, 59, 59, 59, 10, 59, 13, 2...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930..."
...,...,...,...,...
40477,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 3, 10, 50, 13, 18, 8, 59, 59, 59, 59, 59,...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006..."
40478,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 13, 5...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006..."
40479,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 48, 59, 59, 59, 59, 59, 59, 59, 8, 37, 59...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006..."
40480,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006..."


In [65]:
def handle_predicate(pred):
    if len(pred) > 1:
        return pred[:2]
    elif len(pred) == 2:
        return pred.append(0)
    
for i,r in advanced_train.iterrows():
    advanced_train.at[i,'input_ids'], advanced_train.at[i,'predicate'], advanced_train.at[i,'labels']= find_predicate(r['input_ids'],r['labels'])

for i,r in advanced_test.iterrows():
    advanced_test.at[i,'input_ids'], advanced_test.at[i,'predicate'], advanced_test.at[i,'labels'] = find_predicate(r['input_ids'],r['labels'])
advanced_train['predicate'] = advanced_train['predicate'].apply(handle_predicate)
advanced_test['predicate'] = advanced_test['predicate'].apply(handle_predicate)


# advanced_train['labels'] = pd.Series([[int(label_dict[label]) for label in arg] for arg in advanced_train['labels']])

# advanced_test['labels'] = pd.Series([[int(label_dict[label]) for label in arg] for arg in advanced_test['labels']])

In [66]:
advanced_train

,attention_mask,input_ids,labels,predicate
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2632, 1011, 23564, 2386, 1024, 2137, 274...","[40, 59, 59, 59, 59, 59, 59, 48, 13, 3, 3, 59,...","[3102, 1012]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 13, 59, 59, 59, 3, 59, 59, 59, 59...","[3102, 1012]"
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 13, 59, 5...","[2022, 1012]"
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 48, 59, 59, 59, 59, 10, 59, 13, 2...","[3426, 1012]"
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1031, 2023, 4288, 1997, 1037, 9768, 2930...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5...","[2272, 1012]"
...,...,...,...,...
40477,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 3, 10, 50, 13, 18, 8, 59, 59, 59, 59, 59,...","[2709, 1012]"
40478,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 13, 5...","[2031, 1012]"
40479,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 48, 59, 59, 59, 59, 59, 59, 59, 8, 37, 59...","[4797, 1012]"
40480,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2097, 2196, 2709, 2045, 2153, 1006...","[40, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 5...","[2147, 1012]"


In [67]:
def generate_ngrams(words, predicate, arg, n=5):
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = words[i:i+n]
        ngram_label = arg[i:i+n]
        ngrams.append({'predicate': predicate,
                       'input_ids': ngram,
                       'labels': ngram_label})
    return ngrams

ngram_data = []
for index, row in advanced_test.iterrows():
    ngram_data.extend(generate_ngrams(row['input_ids'], row['predicate'], row['labels']))

ngram_test = pd.DataFrame(ngram_data)

ngram_data = []
for index, row in advanced_train.iterrows():
    ngram_data.extend(generate_ngrams(row['input_ids'], row['predicate'], row['labels']))

ngram_train = pd.DataFrame(ngram_data)


In [68]:
ngram_test

,predicate,input_ids,labels
0,"[22822, 8458]","[101, 2054, 2065, 8224, 22822]","[40, 59, 59, 3, 13]"
1,"[22822, 8458]","[2054, 2065, 8224, 22822, 8458]","[59, 59, 3, 13, 13]"
2,"[22822, 8458]","[2065, 8224, 22822, 8458, 2098]","[59, 3, 13, 13, 13]"
3,"[22822, 8458]","[8224, 22822, 8458, 2098, 2046]","[3, 13, 13, 13, 59]"
4,"[22822, 8458]","[22822, 8458, 2098, 2046, 8224]","[13, 13, 13, 59, 0]"
...,...,...,...
97567,"[2224, 1012]","[1996, 3563, 3314, 1998, 9104]","[59, 59, 59, 59, 59]"
97568,"[2224, 1012]","[3563, 3314, 1998, 9104, 11110]","[59, 59, 59, 59, 3]"
97569,"[2224, 1012]","[3314, 1998, 9104, 11110, 2000]","[59, 59, 59, 3, 59]"
97570,"[2224, 1012]","[1998, 9104, 11110, 2000, 2224]","[59, 59, 3, 59, 13]"


In [69]:
import os
def create_generator(df, folder_path, chunk_size):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    chunk_start = 0
    chunk_end = chunk_size
    chunk_count = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    for i in range(chunk_count):
        chunk = df[chunk_start:chunk_end]
        file_name = f"chunk_{i}.csv"
        chunk.to_csv(os.path.join(folder_path, file_name), index=False)
        chunk_start += chunk_size
        chunk_end += chunk_size

create_generator(ngram_train, 'df_train_folder', chunk_size=50)#df_train, df_test
create_generator(ngram_test, 'df_test_folder', chunk_size=50)

## Loading & final preprocessing

In [72]:
import os
import random
import pandas as pd
import tensorflow as tf

def data_generator(folder_path, epochs = 3, max_len=200):
    def pad(tokens):
        tokens = tokens.replace('[', '').replace(']', '')
        tokens = tf.convert_to_tensor([int(token) for token in tokens.split(',')], dtype=tf.int32)
        if tf.size(tokens) > max_len:
            return tokens[tf.size(tokens)-max_len-1:]
        else:
            padding = tf.constant([0 for _ in range(max_len - tf.size(tokens))], dtype=tf.int32)
            return tf.concat([tokens, padding], axis=0)
    
    def extend(attention):
        attention = [int(x) for x in attention.replace('[', '').replace(']', '').split(',')]
        return tf.convert_to_tensor(attention + [0] * (max_len - len(attention)), dtype=tf.int32)

    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)] * epochs
    random.shuffle(file_paths)
    for file_path in file_paths:
        batch_x = []
        batch_y = []
        with open(file_path, 'r') as f:
            df = pd.read_csv(file_path)
            # print(len(df))
            for _, row in df.iterrows():
                input_ids = pad(row['input_ids'])
                attention_mask = extend(row['attention_mask'])
                batch_x.append( tf.stack([input_ids, attention_mask], axis=1))
                batch_y.append(pad(row['labels']))
                # print( tf.stack([input_ids, attention_mask], axis=1))
        yield tf.convert_to_tensor(batch_x), tf.convert_to_tensor(batch_y)
                

In [271]:
from sklearn.preprocessing import normalize

def data_generator2(folder_path, epochs = 3):
    def tensorify(string):
        string = string.replace('[', '').replace(']', '')
        return [int(tok) for tok in string.split(',')]
        
    def normalize_array(array, min_range=0, max_range=1, maxv=29556 ):
        array = np.array(array)
        minv = 0
        array = (array - minv) / (maxv - minv) * (max_range - min_range) + min_range
        return array
    
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)] * epochs
    random.shuffle(file_paths)

    for file_path in file_paths:
        batch_x = []
        batch_y = []
        i=  0
        with open(file_path, 'r') as f:
            df = pd.read_csv(file_path)
            for _, row in df.iterrows():
                input_ids = tensorify(row['input_ids'])
                predicate = tensorify(row['predicate'])
                window = []
                for input_id in input_ids:
                    
                    window.append(tf.constant([input_id, predicate[0], predicate[1]])) 
                if len(window) == 5:
                    batch_x.append(tf.convert_to_tensor(window))
                    batch_y.append(tensorify(row['labels']))
        if batch_x and batch_y:
            yield tf.convert_to_tensor(normalize_array(batch_x),dtype='float32'), tf.convert_to_tensor(batch_y,dtype='int32')

                

In [201]:
sample = [i for i in data_generator2('df_test_folder',epochs=1)][0]
input_dim = sample[0].shape
num_labels = sample[1].shape[1]
sample

KeyboardInterrupt: 

In [ ]:
input_dim = sample[0].shape
num_labels = sample[1].shape[1]

In [ ]:
np.shape(sample[0])

In [ ]:
sample[0][0]

In [ ]:
pd.read_csv('df_test_folder/chunk_0.csv')

In [129]:
num_labels = len(set(label for sequence in df_train['labels'] for label in sequence))
num_labels

61

## Advanced model loading and training

In [267]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, TimeDistributed
from tf2crf import CRF, ModelWithCRFLoss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def create_lstm_crf(seq_len, feature_dim, num_labels):
    print(f'seq_len:{seq_len}, feature_dim:{feature_dim}, num_labels:{num_labels}')
    input = Input(shape=(seq_len, feature_dim), dtype='float64')
    
    bilstm_layer1 = Bidirectional(LSTM(units=feature_dim, return_sequences=True))
    bilstm_output1 = bilstm_layer1(input)
    
    bilstm_layer2 = Bidirectional(LSTM(units=num_labels, return_sequences=True))
    bilstm_output2 = bilstm_layer2(bilstm_output1)

    dense = TimeDistributed(Dense(num_labels, activation='relu'))#(bilstm_output2)
    outputs = dense(bilstm_output2)
    
    crf = CRF(int(62), dtype='float64')
    output = crf(outputs)
    
    base_model = Model(input, output)
    model = ModelWithCRFLoss(base_model, sparse_target=True)

    return model

seq_len = sample[1].shape[1]
feature_dim = sample[0].shape[2]
num_labels = seq_len #label for every element in the sequence

model = create_lstm_crf(seq_len, feature_dim, num_labels)
model.build((None, int(seq_len), int(feature_dim)))#model.build(input_dim)
model.summary()

seq_len:5, feature_dim:3, num_labels:5
Model: "model_with_crf_loss_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_22 (Functional)       ((None, 5),               4919      
                              (None, 5, 62),                     
                              (None,),                           
                              (62, 62))                          
                                                                 
Total params: 4923 (34.25 KB)
Trainable params: 4919 (34.23 KB)
Non-trainable params: 4 (16.00 Byte)
_________________________________________________________________


In [268]:
epochs = 3
loss = SparseCategoricalCrossentropy()

In [269]:
model.compile(optimizer=Adam(0.001))#,loss=loss)

In [270]:
if input("type 'load' to load the model from checkpoint files and skip training, else press enter to start training")  != 'load':
    model.fit(data_generator2('df_train_folder', epochs=int(epochs)), batch_size=int(50), epochs=int(epochs), steps_per_epoch=int(20000))#809
    model.save_weights('./advanced_model4')
else:
    model.load_weights('./advanced_model4')


type 'load' to load the model from checkpoint files and skip training, else press enter to start training 


Epoch 1/3
20000/20000 [==============================] - 703s 34ms/step - loss: 3.1094 - accuracy: 0.8745
Epoch 2/3
20000/20000 [==============================] - 698s 35ms/step - loss: 2.8249 - accuracy: 0.8813
Epoch 3/3
19747/20000 [============================>.] - ETA: 8s - loss: 2.6916 - accuracy: 0.8842

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
NameError: name 'mx' is not defined
Traceback (most recent call last):

  File "C:\Users\katko\miniconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\katko\miniconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\katko\miniconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\katko\miniconda3\Lib\site-packages\keras\src\engine\data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\katko\AppData\Local\Temp\ipykernel_30772

## Evaluation

In [ ]:
xy_tensors = [i for i in data_generator2('df_test_folder')]
x_tensors, y_tensors = [i[0] for i in xy_tensors], [i[1] for i in xy_tensors]

if len(x_tensors[-1]) != 50:
    x_tensors.pop() 
    y_tensors.pop()

x_tensors_filtered = [tensor for tensor in x_tensors if tensor.shape[0] == 50]
y_tensors_filtered = [tensor for tensor in y_tensors if tensor.shape[0] == 50]

x = tf.convert_to_tensor(x_tensors_filtered)
y = tf.convert_to_tensor(y_tensors_filtered)
x = tf.reshape(x, [-1, 200, 2])
y = tf.reshape(y, [-1, 200])

In [ ]:
x = tf.reshape(x, [-1, 5, 3])
y = tf.reshape(y, [-1, 5])

In [ ]:
output = model.predict(x[:50])

In [ ]:
output[0]

In [ ]:
y[0]